<img src="https://iteso.mx/documents/27014/202031/Logo-ITESO-MinimoH.png"
     align="right"
     width="300"/>

# LSTM para series de tiempo
## *Modelos no lineales para pronósitico*  - Pedro Martinez

---

# Objetivo

Diseñar una red LSTM para predecir el comportamiento del índice S&P 500 utilizando datos reales de Yahoo Finance.

Además de construir el modelo, analizaremos el **efecto de los principales hiperparámetros** de la capa LSTM en Keras. Considerando:


1. Descargar y preparar datos financieros reales.  
2. Construir una red LSTM paso a paso.  
3. Comprender el papel de los hiperparámetros:  
    `units`, `return_sequences`, `dropout`, `recurrent_dropout`, `batch_size`, `epochs`, `optimizer`.  
4. Visualizar cómo cambian los resultados al modificarlos.

In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# Descargar datos del S&P 500
data = yf.download('^GSPC', start='2018-01-01', end='2025-01-01')

# Aplanar columnas en caso de que vengan con MultiIndex
if isinstance(data.columns, pd.MultiIndex):
    data.columns = [' '.join(col).strip() for col in data.columns.values]

# Verificamos si existe la columna 'Close'
if 'Close' in data.columns:
    close_series = data['Close']
else:
    # En algunos casos se llama 'Close ^GSPC'
    col_close = [c for c in data.columns if 'Close' in c][0]
    close_series = data[col_close]

# Asegurar formato 1D numérico
close_series = pd.Series(close_series.values, index=pd.to_datetime(data.index))
close_series = pd.to_numeric(close_series, errors='coerce').dropna()

# Gráfico interactivo con Plotly
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=close_series.index,
    y=close_series.values,
    mode='lines',
    line=dict(color='royalblue'),
    name='Cierre S&P 500'
))

fig.update_layout(
    title='S&P 500 — Precio de Cierre Diario (2018–2024)',
    xaxis_title='Fecha',
    yaxis_title='Precio (USD)',
    hovermode='x unified',
    yaxis=dict(tickprefix='$', separatethousands=True)
)

fig.show()


/tmp/ipython-input-1869488872.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download('^GSPC', start='2018-01-01', end='2025-01-01')
[*********************100%***********************]  1 of 1 completed


1. Escalamiento de los datos.
Cual escalador podemos utilizar?

2. Definir entrada de datos. Como se pueden ingresar los datos a un LSTM?

3. Como se declara un modelos LSTM en Keras?